In [ ]:
%%capture
!pip install vllm
!pip install bitsandbytes>=0.46.1
# !pip install --upgrade --no-deps vllm

In [ ]:
!pip install gdown

In [ ]:
!mkdir -p data
!gdown 1EiJmjVmq54zrH2n2h9yUbpktkJKlYwKF -O data/alqac25_train.json
!gdown 1GUbdSU-ls_THuU-LStkFR6nA3dEdaG2X -O data/alqac25_law.json

## load model

In [ ]:
import os
from vllm import LLM, SamplingParams

max_seq_length = 8192
# think_model_name = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"
think_model_name = "unsloth/DeepSeek-R1-0528-Qwen3-8B-unsloth-bnb-4bit"
# think_model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"
think_model = LLM(
        model=think_model_name,
        gpu_memory_utilization=0.99,
        max_model_len = max_seq_length,
        tensor_parallel_size=1,
    )

In [ ]:
# max_input_tokens = len(tokenizer(prompt)["input_ids"])
# max_new_tokens = max_seq_length - max_input_tokens

# max_new_tokens = 2048
max_new_tokens = 4096
sampling_params = SamplingParams(
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    max_tokens=max_new_tokens,
)

In [ ]:
def generate_think_response(history: list[dict]) -> str: 
    response = think_model.chat(messages=history, sampling_params=sampling_params)
    response_text = response[0].outputs[0].text
    return response_text

system_prompt = """Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm vụ trả lời các câu hỏi ngắn gọn, chính xác, dựa trên nội dung điều luật được cung cấp. Chỉ trả lời bằng Tiếng Việt.

Bạn hãy trả lời theo định dạng sau:
<think>
[Suy nghĩ, phân tích của bạn]
</think>
[Câu trả lời của bạn]
"""
prompt = '''Đáp án cuối cùng sau khi suy luận đã có, hãy phân tích để giải thích cho kết luận: '10 ngày' với câu hỏi dạng Tự luận.

Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.

Loại câu hỏi: Tự luận

Câu hỏi: Trong trường hợp các bên không có thỏa thuận khác hoặc quy tắc tố tụng của trung tâm trọng tài không có quy định khác, Trung tâm trọng tài phải gửi cho bị đơn bản sao đơn khởi kiện của nguyên đơn và những tài liệu theo quy định trong thời hạn bao lâu kể từ ngày nhận được đơn khởi kiện?

Bối cảnh: 
Thông báo đơn khởi kiện

Nếu các bên không có thoả thuận khác hoặc quy tắc tố tụng của Trung tâm trọng tài không có quy định khác, trong thời hạn 10 ngày, kể từ ngày nhận được đơn khởi kiện, các tài liệu kèm theo và chứng từ nộp tạm ứng phí trọng tài, Trung tâm trọng tài phải gửi cho bị đơn bản sao đơn khởi kiện của nguyên đơn và những tài liệu theo quy định tại khoản 3 Điều 30 của Luật này.

Hãy đưa ra câu trả lời theo format sau:
1. Phân tích câu hỏi: [trình bày ngắn gọn nội dung và ý định của câu hỏi]
2. Dẫn chứng từ bối cảnh:
   - Hãy tách từng đoạn dài trong bối cảnh thành nhiều ý nhỏ rõ ràng.
   - Hãy tách ít nhất 3 đến 5 ý ở phần dẫn chứng từ bối cảnh
   - Mỗi ý nên nêu rõ nội dung pháp lý, viết ngắn gọn dễ hiểu.
   - Ví dụ:
       - [ý 1 từ đoạn luật A]
       - [ý 2 từ đoạn luật A]
       - [ý 3 từ đoạn luật B]
   - Ghi rõ đoạn nào có liên quan đến câu hỏi.
3. Suy luận step-by-step:
   a) [bước suy luận 1 dựa trên dẫn chứng ở trên]
   b) [bước suy luận 2 tiếp theo]
   …
4. Kết luận: 10 ngày
'''

messages = [{'role': 'system', 
             'content': system_prompt}, 
            {'role': 'user', 
             'content': prompt}]
response_text = generate_think_response(messages)
print(response_text)

In [ ]:
response_text.find('<think>'), response_text.find('</think>')

## process data

In [ ]:
import json

# Load data
with open('data/alqac25_train.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

with open('data/alqac25_law.json', 'r', encoding='utf-8') as f:
    corpus = json.load(f)

#Build lookup dictionary: { law_id: { article_id: text } }
corpus_dict = {}
for law in corpus:
    law_id = law['id']
    articles_dict = {article['id']: article['text'] for article in law.get('articles', [])}
    corpus_dict[law_id] = articles_dict

# Replace relevant_articles in questions
for q in questions:
    new_articles = []
    for ref in q.get('relevant_articles', []):
        law_id = ref['law_id']
        article_id = ref['article_id']
        text = corpus_dict.get(law_id, {}).get(article_id)
        if text:
            new_articles.append(text)
    q['relevant_articles'] = new_articles  # Replace with list of strings

# Save the updated file
with open('data/alqac25_train_question_text.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=2)

In [ ]:
import json
with open('data/alqac25_train_question_text.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

In [ ]:
questions[0]

In [ ]:
system_prompt = """Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm vụ trả lời các câu hỏi ngắn gọn, chính xác, dựa trên nội dung điều luật được cung cấp. Chỉ trả lời bằng Tiếng Việt.

Bạn hãy trả lời theo định dạng sau:
<think>
[Suy nghĩ, phân tích của bạn]
</think>
[Câu trả lời của bạn]
"""
def build_prompt(question: dict):
    question_text, article_texts, question_type = question['text'], question['relevant_articles'], question['question_type']
    texts = [a for a in article_texts if isinstance(a, str)]
    article_content = "\n\n".join(texts)
    if question_type == "Tự luận":
        prompt = (
            "Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\n"
            f"Loại câu hỏi: Tự luận\n\n"
            f"Câu hỏi: {question_text}\n\n"
            f"Bối cảnh: \n{article_content}\n\n"
            "Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. "
            "Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.\n"
            "Không được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.\n\n"
            "<think>\n"
            "1. Phân tích câu hỏi: [trình bày ngắn gọn nội dung và ý định của câu hỏi]\n"
            "2. Dẫn chứng từ bối cảnh:\n"
            "   - Hãy tách từng đoạn dài trong bối cảnh thành nhiều ý nhỏ rõ ràng.\n"
            "   - Hãy tách ít nhất 3 đến 5 ý ở phần dẫn chứng từ bối cảnh\n"
            "   - Mỗi ý nên nêu rõ nội dung pháp lý, viết ngắn gọn dễ hiểu.\n"
            "   - Ví dụ:\n"
            "       - [ý 1 từ đoạn luật A]\n"
            "       - [ý 2 từ đoạn luật A]\n"
            "       - [ý 3 từ đoạn luật B]\n"
            "   - Ghi rõ đoạn nào có liên quan đến câu hỏi.\n"
            "3. Suy luận step-by-step:\n"
            "   a) [bước suy luận 1 dựa trên dẫn chứng ở trên]\n"
            "   b) [bước suy luận 2 tiếp theo]\n"
            "   …\n"
            "4. Kết luận: [tóm tắt câu trả lời cuối cùng dựa trên suy luận]\n"
            "</think>\n\n"
            "[Kết luận cuối cùng sau khi suy luận]"
        )
    elif question_type == "Trắc nghiệm":
        prompt = (
            "Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\n"
            f"Loại câu hỏi: Trắc nghiệm (format của Kết luận cuối cùng sau khi suy luận là 1 trong 4 kết luận: 'A', 'B', 'C', 'D'. Không được giải thích gì thêm.)\n\n"
            f"Câu hỏi: {question_text}\n\n"
            f"Bối cảnh: \n{article_content}\n\n"
            f"4 lựa chọn: \n"
            f"A: {q['choices']['A']}\n"
            f"B: {q['choices']['B']}\n"
            f"C: {q['choices']['C']}\n"
            f"D: {q['choices']['D']}\n"
            "\n\n"
            "Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. "
            "Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.\n"
            "Không được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.\n\n"
            "<think>\n"
            "1. Phân tích câu hỏi: [trình bày ngắn gọn nội dung và ý định của câu hỏi]\n"
            "2. Dẫn chứng từ bối cảnh:\n"
            "   - Hãy tách từng đoạn dài trong bối cảnh thành nhiều ý nhỏ rõ ràng.\n"
            "   - Hãy tách ít nhất 3 đến 5 ý ở phần dẫn chứng từ bối cảnh\n"
            "   - Mỗi ý nên nêu rõ nội dung pháp lý, viết ngắn gọn dễ hiểu.\n"
            "   - Ví dụ:\n"
            "       - [ý 1 từ đoạn luật A]\n"
            "       - [ý 2 từ đoạn luật A]\n"
            "       - [ý 3 từ đoạn luật B]\n"
            "   - Ghi rõ đoạn nào có liên quan đến câu hỏi.\n"
            "3. Suy luận step-by-step:\n"
            "   a) [bước suy luận 1 dựa trên dẫn chứng ở trên]\n"
            "   b) [bước suy luận 2 tiếp theo]\n"
            "   …\n"
            "4. Kết luận: [tóm tắt câu trả lời cuối cùng dựa trên suy luận]\n"
            "</think>\n\n"
            "[Kết luận cuối cùng sau khi suy luận]"
        )
    elif question_type == "Đúng/Sai":
            prompt = (
            "Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\n"
            f"Loại câu hỏi: Đúng/Sai (format của Kết luận cuối cùng sau khi suy luận là 1 trong 2 kết luận: 'Đúng', 'Sai'. Không được giải thích gì thêm.)\n\n"
            f"Câu hỏi: {question_text}\n\n"
            f"Bối cảnh: \n{article_content}\n\n"
            "Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. "
            "Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.\n"
            "Không được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.\n\n"
            "<think>\n"
            "1. Phân tích câu hỏi: [trình bày ngắn gọn nội dung và ý định của câu hỏi]\n"
            "2. Dẫn chứng từ bối cảnh:\n"
            "   - Hãy tách từng đoạn dài trong bối cảnh thành nhiều ý nhỏ rõ ràng.\n"
            "   - Hãy tách ít nhất 3 đến 5 ý ở phần dẫn chứng từ bối cảnh\n"
            "   - Mỗi ý nên nêu rõ nội dung pháp lý, viết ngắn gọn dễ hiểu.\n"
            "   - Ví dụ:\n"
            "       - [ý 1 từ đoạn luật A]\n"
            "       - [ý 2 từ đoạn luật A]\n"
            "       - [ý 3 từ đoạn luật B]\n"
            "   - Ghi rõ đoạn nào có liên quan đến câu hỏi.\n"
            "3. Suy luận step-by-step:\n"
            "   a) [bước suy luận 1 dựa trên dẫn chứng ở trên]\n"
            "   b) [bước suy luận 2 tiếp theo]\n"
            "   …\n"
            "4. Kết luận: [tóm tắt câu trả lời cuối cùng dựa trên suy luận]\n"
            "</think>\n\n"
            "[Kết luận cuối cùng sau khi suy luận]"
        )
    else:
        print('Lỗi')

    return prompt

for q in questions:
    q['system_prompt'] = system_prompt
    q['prompt'] = build_prompt(q)
    gemini_text = f"Kết luận cuối cùng sau khi suy luận đã có, hãy phân tích để giải thích cho kết luận: '{q['answer']}' với câu hỏi dạng {q['question_type']}.\n\n"
    q['think_prompt'] = gemini_text + q['prompt'].replace('[Kết luận cuối cùng sau khi suy luận]', q['answer']).replace('[tóm tắt câu trả lời cuối cùng dựa trên suy luận]', q['answer'])
    # print(q['think_prompt'].find('Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.\nKhông được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.\n\n<think>'))
    q['think_prompt'] = q['think_prompt'].replace('Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.\nKhông được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.\n\n<think>','Hãy đưa ra câu trả lời theo format sau:').split('</think>')[0]
    q['think_prompt'] = q['think_prompt'].replace("(format của Kết luận cuối cùng sau khi suy luận là 1 trong 4 kết luận: 'A', 'B', 'C', 'D'. Không được giải thích gì thêm.)",'')
    q['think_prompt'] = q['think_prompt'].replace("(format của Kết luận cuối cùng sau khi suy luận là 1 trong 2 kết luận: 'Đúng', 'Sai'. Không được giải thích gì thêm.)",'')

with open('data/alqac25_train_question_text_new.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=2)

In [ ]:
with open('data/alqac25_train_question_text_new.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

In [ ]:
# (questions[0]['prompt'])

In [ ]:
# print(questions[50]['think_prompt'])

In [ ]:
# from pprint import pprint
# pprint(questions[0]['prompt'])

## gen think answer

In [ ]:
# !rm -rf /kaggle/working/data_train_extracted/*

In [ ]:
import json
import os
import logging
import random
import re
import threading
import queue
from google import genai
from google.genai import types
from glob import glob
import time
from tqdm import tqdm

# === CẤU HÌNH ===
INPUT_FILE = 'data/alqac25_train_question_text_new.json'
OUTPUT_DIR = 'data_train_extracted'

# === LOGGING ===
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load JSON dataset
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    samples = json.load(f)

# Thread-safe queue
sample_queue = queue.Queue()
for sample in samples:
    sample_queue.put(sample)

# # === MODEL CALL ===
def generate_think_response(history: list[dict]) -> str: 
    response = think_model.chat(messages=history, sampling_params=sampling_params)
    response_text = response[0].outputs[0].text
    return response_text

# === CHẠY BÌNH THƯỜNG KHÔNG THREAD ===
def run_single_thread():
    i = 0
    for sample in tqdm(samples):
        i += 1
        # if i > 5:
        #     break
        try:
            sid = sample["question_id"]
            out_path = os.path.join(OUTPUT_DIR, f"{sid}.json")
    
            # ✅ Bỏ qua nếu file đã tồn tại
            if os.path.exists(out_path): #########################
            # if os.path.exists(out_path) or out_path.split('/')[-1] in runned_files:
                continue
    
            history = [
                {"role": "system", "content": sample["system_prompt"]},
                {"role": "user", "content": sample["think_prompt"]}
            ]
            # print('historyyyyyyyyyyy',i,history)

            try:
                # answer = generate_gemini_response(api_key, history, logger)
                answer = generate_think_response(history)
                match = re.search(r'<think>(.*?)</think>', answer, re.DOTALL) # re.DOTALL để khớp cả ký tự xuống dòng
                # print(answer)
                if match:
                    # think_content = "<think>" + match.group(1) + "</think>" # Thêm lại thẻ <think> và </think> nếu bạn muốn
                    think_content = answer.split('</think>')[-1]
                else:
                    print(f'❌ Lỗi <think> trong file {sample["question_id"]} với answer là: {answer}')
                    continue
                processed_answer = '<think>\n' + think_content + '\n</think>\n' +sample['answer']
                
            except Exception as e:
                print(f"[{sid}] Lỗi khi gọi model: {e}")
                continue  # ❌ Bỏ qua nếu lỗi
                
            # print('answer', answer)
            # print('processed_answer', processed_answer)
    
            # ✅ Lưu đủ 6 key: id, text, relevant_articles, system_prompt, prompt, answer
            output_data = {
                "question_id": sample["question_id"],
                "question_type": sample["question_type"],
                "text": sample["text"],
                "relevant_articles": sample["relevant_articles"],
                "answer": sample["answer"],
                "system_prompt": sample["system_prompt"],
                "prompt": sample["prompt"],
                "answer_think": processed_answer,
            }
    
            with open(out_path, 'w', encoding='utf-8') as out_f:
                json.dump(output_data, out_f, ensure_ascii=False, indent=2)
            print(f'Đã lưu file thứ {i}: {out_path}')
            
        except Exception as e:
            print(f'❌ Lỗi trong file {sample["question_id"]}: {e}')

    logger.info(f"✅ Hoàn thành tuần tự. Kết quả lưu tại: {OUTPUT_DIR}")

# === MAIN ===
if __name__ == "__main__":
    run_single_thread()

In [ ]:
import json

# === CẤU HÌNH ===
INPUT_FILE = 'data/alqac25_train_question_text_new.json'
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    samples = json.load(f)
len(samples)

## merge json files (nhớ lấy các lựa chọn trắc nghiệm)

In [ ]:
import os
import json

def combine_json_files(input_directory, output_file_name):
    """
    Gộp tất cả các file JSON trong một thư mục thành một file JSON duy nhất.

    Args:
        input_directory (str): Đường dẫn đến thư mục chứa các file JSON.
        output_file_name (str): Tên của file JSON đầu ra.
    """
    all_data = []
    
    # Kiểm tra xem thư mục đầu vào có tồn tại không
    if not os.path.isdir(input_directory):
        print(f"Lỗi: Thư mục '{input_directory}' không tồn tại.")
        return

    # Duyệt qua tất cả các file trong thư mục
    for filename in os.listdir(input_directory):
        if filename.endswith(".json"):
            filepath = os.path.join(input_directory, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    all_data.append(data)
                print(f"Đã đọc file: {filename}")
            except json.JSONDecodeError as e:
                print(f"Lỗi đọc JSON từ file '{filename}': {e}")
            except Exception as e:
                print(f"Lỗi không xác định khi xử lý file '{filename}': {e}")

    # Ghi tất cả dữ liệu vào một file JSON duy nhất
    if all_data:
        try:
            with open(output_file_name, 'w', encoding='utf-8') as f:
                json.dump(all_data, f, ensure_ascii=False, indent=2)
            print(f"Đã gộp thành công {len(all_data)} file vào '{output_file_name}'.")
        except Exception as e:
            print(f"Lỗi khi ghi file đầu ra '{output_file_name}': {e}")
    else:
        print(f"Không tìm thấy file JSON nào trong thư mục '{input_directory}' để gộp.")

# Ví dụ sử dụng:
# Giả sử các file JSON của bạn nằm trong thư mục có tên 'data_zalo_extracted'
input_folder = "data_train_extracted" 
output_json_file = "data_train_legal_qa.json"

combine_json_files(input_folder, output_json_file)

In [ ]:
import json

# Define file paths
main_input_file = 'data_train_legal_qa.json'
source_choices_file = 'data/alqac25_train_question_text_new.json'
output_file = 'data_train_legal_qa_new.json'

try:
    # 1. Load the main data file
    with open(main_input_file, 'r', encoding='utf-8') as f:
        main_data = json.load(f)
    print(f"Đã đọc {len(main_data)} mục từ file '{main_input_file}'.")

    # 2. Load the source file containing choices
    with open(source_choices_file, 'r', encoding='utf-8') as f:
        source_data = json.load(f)
    print(f"Đã đọc {len(source_data)} mục từ file '{source_choices_file}'.")

    # 3. Create a lookup dictionary for choices from the source data
    choices_lookup = {}
    for item in source_data:
        if item.get('question_type') == "Trắc nghiệm" and 'choices' in item:
            choices_lookup[item['question_id']] = item['choices']
    print(f"Đã tạo lookup dictionary với {len(choices_lookup)} mục lựa chọn.")

    # 4. Iterate through the main data and add choices
    merged_count = 0
    for item in main_data:
        if item.get('question_type') == "Trắc nghiệm":
            question_id = item.get('question_id')
            if question_id in choices_lookup:
                if 'choices' not in item: # Only add if 'choices' key doesn't exist
                    item['choices'] = choices_lookup[question_id]
                    merged_count += 1
                else:
                    print(f"Cảnh báo: Mục '{question_id}' đã có khóa 'choices'. Bỏ qua.")
            else:
                print(f"Cảnh báo: Không tìm thấy lựa chọn cho 'Trắc nghiệm' question_id: {question_id} trong file nguồn.")

    # 5. Save the modified data to a new JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(main_data, f, indent=2, ensure_ascii=False)

    print(f"\nĐã hợp nhất 'choices' cho {merged_count} mục loại 'Trắc nghiệm'.")
    print(f"Dữ liệu đã được lưu vào file mới: '{output_file}'")

except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy file {e.filename}. Vui lòng kiểm tra lại đường dẫn.")
except json.JSONDecodeError as e:
    print(f"Lỗi: Không thể giải mã JSON từ file. Đảm bảo file có định dạng JSON hợp lệ. Chi tiết: {e}")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn: {e}")

### => file data_train_legal_qa_new.json là file dùng để training